# Seq2Seq Chatbot
# How to implement Seq2Seq LSTM Model in Keras #ShortcutNLP

In [34]:
# https://towardsdatascience.com/how-to-implement-seq2seq-lstm-model-in-keras-shortcutnlp-6f355f3e5639

In [36]:
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [37]:
from nltk.tokenize import sent_tokenize
import numpy as np
import re

#lines= pd.read_table('jpn.txt', names=['eng', 'prediction'])
data_path = '../../Dataset/Bangladesh Dummy dataset from wikipedia.txt'

with open(data_path,'r', encoding='utf-8') as f:
    lines = f.read()

lines = " ".join(lines.split())
lines = re.sub(r"\s+", " ", lines)

sentences = sent_tokenize(lines)
sent = np.asarray(sentences)
sentTwo = sent.reshape(len(sent)//2,2)


In [38]:
lines = lines.replace('\n', ' ')
lines

'Bangladesh officially the People\'s Republic of Bangladesh (গণপ্রজাতন্ত্রী বাংলাদেশ Gônoprojatontri Bangladesh), is a country in South Asia. While it is the 92nd-largest country, spanning 147,570 square kilometres (56,980 sq mi), it is the world\'s 8th-most populous country with a population nearing 163 million, making it one of the most densely populated countries in the world. Bangladesh shares land borders with India to the west, north and the east and Myanmar to the east, whereas the Bay of Bengal lies to its south. Dhaka, its capital and largest city, is also the economic, political and the cultural hub of the country. Chittagong, the largest sea port, is the second largest city. The country\'s geography is dominated by the Ganges delta which empties into the Bay of Bengal the combined waters of several river systems, including those of the Brahmaputra and the Ganges. As a result, the country is criss-crossed by numerous rivers and inland water ways. Highlands with evergreen fore

In [39]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [40]:
col1, col2 = create_dataset(sentTwo)
#col1 = col1.reshape(-1)
col2 = col2.reshape(-1,1)
#col2.shape
dt = np.concatenate((col1,col2), axis=1)


In [41]:
import pandas as pd

lines = pd.DataFrame(dt, columns=['eng','prediction'])

In [42]:
lines.tail(10)
#lines.prediction.tail(5)

,eng,prediction
460,its started in 1963 at Dhaka.,Federation of Film Societies of Bangladesh is ...
461,Federation of Film Societies of Bangladesh is ...,Museums and libraries Main articles: Museums i...
462,Museums and libraries Main articles: Museums i...,"The Ahsan Manzil, the former residence of the ..."
463,"The Ahsan Manzil, the former residence of the ...",The Tajhat Palace Museum preserves artefacts o...
464,The Tajhat Palace Museum preserves artefacts o...,The Ethnological Museum of Chittagong showcase...
465,The Ethnological Museum of Chittagong showcase...,The Liberation War Museum documents the Bangla...
466,The Liberation War Museum documents the Bangla...,The Hussain Shahi dynasty established royal li...
467,The Hussain Shahi dynasty established royal li...,The trend of establishing libraries continued ...
468,The trend of establishing libraries continued ...,The Northbrook Hall Public Library was establi...
469,The Northbrook Hall Public Library was establi...,The Great Bengal Library Association was forme...


In [43]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.prediction=lines.prediction.apply(lambda x: x.lower())

# to install mecab
# sudo apt install mecab mecab-ipadic-utf8
#import MeCab
#wakati = MeCab.Tagger("-Owakati")
#lines.prediction = lines.prediction.apply(lambda x: wakati.parse(x).strip("\n"))

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.prediction=lines.prediction.apply(lambda x: re.sub("'", '', x))
exclude = set(string.punctuation) # Set of all special characters

# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.prediction=lines.prediction.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
#remove_digits = str.maketrans('', '', digits)
#lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
#lines.prediction = lines.prediction.apply(lambda x: re.sub("[123456789]", "", x))
# Remove extra spaces

lines.eng=lines.eng.apply(lambda x: x.strip())
lines.prediction=lines.prediction.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.prediction=lines.prediction.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.prediction = lines.prediction.apply(lambda x : 'START_ '+ x + ' _END')
lines.head(10)
lines.tail(10)

,eng,prediction
460,its started in 1963 at dhaka,START_ federation of film societies of banglad...
461,federation of film societies of bangladesh is ...,START_ museums and libraries main articles mus...
462,museums and libraries main articles museums in...,START_ the ahsan manzil the former residence o...
463,the ahsan manzil the former residence of the n...,START_ the tajhat palace museum preserves arte...
464,the tajhat palace museum preserves artefacts o...,START_ the ethnological museum of chittagong s...
465,the ethnological museum of chittagong showcase...,START_ the liberation war museum documents the...
466,the liberation war museum documents the bangla...,START_ the hussain shahi dynasty established r...
467,the hussain shahi dynasty established royal li...,START_ the trend of establishing libraries con...
468,the trend of establishing libraries continued ...,START_ the northbrook hall public library was ...
469,the northbrook hall public library was establi...,START_ the great bengal library association wa...


In [44]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of prediction 
all_prediction_words=set()
for prediction in lines.prediction:
    for word in prediction.split():
        if word not in all_prediction_words:
            all_prediction_words.add(word)

In [45]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

96

In [46]:
# Max Length of target sequence
lenght_list=[]
for l in lines.prediction:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

98

In [47]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_prediction_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_prediction_words)
num_encoder_tokens, num_decoder_tokens

(2777, 2777)

In [48]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

2778

In [49]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [50]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [66]:
lines = shuffle(lines)
lines.head(10)
input_token_index

{'031': 1,
 '033': 2,
 '036': 3,
 '04': 4,
 '044': 5,
 '058': 6,
 '095': 7,
 '1': 8,
 '10': 9,
 '1000': 10,
 '10000': 11,
 '1053': 12,
 '106': 13,
 '107': 14,
 '10927': 15,
 '10th': 16,
 '11': 17,
 '11000': 18,
 '113000': 19,
 '11th': 20,
 '12': 21,
 '120': 22,
 '1281500': 23,
 '1296': 24,
 '12th': 25,
 '13': 26,
 '13000mile': 27,
 '1303': 28,
 '135000': 29,
 '136th': 30,
 '138': 31,
 '139': 32,
 '14': 33,
 '140000': 34,
 '143th': 35,
 '144': 36,
 '148': 37,
 '1487': 38,
 '14th': 39,
 '15': 40,
 '1531300': 41,
 '157': 42,
 '16': 43,
 '1611': 44,
 '1629': 45,
 '162951560': 46,
 '1666': 47,
 '17': 48,
 '1700': 49,
 '1700s': 50,
 '1757': 51,
 '1776': 52,
 '178': 53,
 '1785000': 54,
 '18': 55,
 '185128': 56,
 '1857': 57,
 '1858': 58,
 '1862': 59,
 '1879': 60,
 '1882': 61,
 '18th': 62,
 '19': 63,
 '190': 64,
 '1905': 65,
 '1906': 66,
 '1913': 67,
 '191329': 68,
 '1931': 69,
 '1935': 70,
 '1937': 71,
 '193814': 72,
 '194': 73,
 '1940': 74,
 '1943': 75,
 '1946': 76,
 '1947': 77,
 '1949': 78,


In [52]:
# Train - Test Split
X, y = lines.eng, lines.prediction
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((423,), (47,))

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [53]:
X_train.to_pickle('Weights/X_train1.pkl')
X_test.to_pickle('Weights/X_test1.pkl')

In [54]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder - Decoder Model Architecture

In [55]:
latent_dim = 50

In [56]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [57]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [58]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [25]:
#from IPython.display import Image
#Image(retina=True, filename='train_model.png')

In [64]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 47
epochs = 50

validation_steps = val_samples//batch_size
validation_steps

1

In [65]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

#model.fit([encoder_inputs, decoder_inputs], decoder_outputs,
#          batch_size=batch_size,
#          epochs=epochs,
#          validation_split=0.2)

Epoch 1/50
8/9 [=========================>....] - ETA: 0s - loss: 1.6203

InvalidArgumentError: indices[6,2] = 2777 is not in [0, 2777)
	 [[{{node embedding_3/embedding_lookup}}]]

### Always remember to save the weights

In [ ]:
model.save_weights('nmt_weights.h5')

### Load the weights, if you close the application

In [ ]:
model.load_weights('nmt_weights.h5')

### Inference Setup

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
Image(retina=True, filename='encoder_model.png')

In [ ]:
Image(retina=True, filename='decoder_model.png')

### Decode sample sequeces

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Evaluation on Train Dataset

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual prediction Sentence:', y_train[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

### Evaluation on Validation Dataset

In [ ]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual prediction Sentence:', y_test[k:k+1].values[0][6:-4])
print('Predicted prediction Sentence:', decoded_sentence[:-4])